In [30]:
#provides a finer price detail 
#instead of using 4 hour candle data we can also use the five minute candle data

In [31]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [32]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [33]:
pair = "USD_JPY"

In [34]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [35]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [36]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

In [37]:
df_trades["next"] = df_trades["time"].shift(-1)

In [38]:
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades["trade_start"] = df_trades.time + dt.timedelta(hours=4)

In [39]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [45]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0

def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)


def process_m5(m5_df, row):
    result = 0.0
    for index, price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            print(f"Signal at {index} {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}")
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values, row.ENTRY)
            break
    return result

In [46]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    # print(f"{row.time} {signal_text(row.SIGNAL)} {row.ENTRY:.2f} {m5_data.shape}")
    total += process_m5(m5_data, row)
    #if index > 10:
    #    break
print(total)

Signal at 102 108.17 108.65 -1.00
Signal at 54 108.56 108.51 1.00
Signal at 235 108.55 108.55 1.00
Signal at 20 108.77 108.81 -1.00
Signal at 62 109.46 109.45 1.00
Signal at 15 109.67 109.62 1.00
Signal at 37 109.56 109.58 -1.00
Signal at 45 109.78 109.77 1.00
Signal at 110 109.23 109.24 -1.00
Signal at 111 109.11 109.22 -1.00
Signal at 44 108.73 108.73 -1.00
Signal at 75 109.69 109.64 1.00
Signal at 290 110.02 110.02 1.00
Signal at 83 109.66 109.70 -1.00
Signal at 39 110.58 110.57 1.00
Signal at 25 110.61 110.59 1.00
Signal at 45 110.70 110.74 -1.00
Signal at 13 110.63 110.65 -1.00
Signal at 63 110.39 110.39 -1.00
Signal at 62 112.03 112.01 1.00
Signal at 12 112.01 112.00 1.00
Signal at 111 111.54 111.56 -1.00
Signal at 42 111.45 111.46 -1.00
Signal at 129 111.24 111.24 1.00
Signal at 110 111.34 111.33 1.00
Signal at 188 111.46 111.43 1.00
Signal at 46 111.89 111.86 1.00
Signal at 63 111.40 111.45 -1.00
Signal at 157 111.45 111.45 -1.00
Signal at 43 111.27 111.27 -1.00
Signal at 251 1